In [15]:
import os
import sys
sys.path.append('/'.join(os.getcwd().split('/')[:-1]))
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torchtext
from konlpy.tag import Mecab
from torchtext.data import Field, BucketIterator, TabularDataset
from model.bidirec_LSTM import bidirec_LSTM
from collections import defaultdict

In [16]:
USE_CUDA = torch.cuda.is_available()
DEVICE = 0 if USE_CUDA else -1

In [17]:
batch_size = 64
# Tokenizer
tagger = Mecab()
tagger = tagger.morphs

# Make Field
REVIEW = Field(tokenize=tagger, use_vocab=True, lower=True, #init_token="<s>", eos_token="</s>", 
               include_lengths=True, batch_first=True)
LABEL = Field(sequential=False, use_vocab=False, preprocessing=lambda x: int(x))

# Get train/test data
train_data, test_data = TabularDataset.splits(
                   path="../data/", train='train_docs.txt', validation="test_docs.txt",
                   format='tsv', fields=[('review', REVIEW), ('label', LABEL)])

# Build Vocaburary
REVIEW.build_vocab(train_data)
len(REVIEW.vocab)

# Make iterator for splits
train_iter, test_iter = BucketIterator.splits(
    (train_data, test_data), batch_size=batch_size, device=DEVICE, # device -1 : cpu, device 0 : 남는 gpu
    sort_key=lambda x: len(x.review), sort_within_batch=True, repeat=False) # x.TEXT 길이 기준으로 정렬

In [18]:
V = len(REVIEW.vocab)
D = 100
H = 200
H_f = 1000
O = 1
da = 300
# r = 5
# num_layers = 3
num_directions = 2
bidirec = True
batch_size = 64
# weight_decay_rate = 0.0001
LR = 0.01
STEP = 10

In [19]:
# model_path = '../model/self_attn_1H_r5.model'
model_path = '../model/self_attn_1H_r20.model'  
# model_path = '../model/self_attn_3H_r5.model' 
num_layers = int(model_path.split('_')[2][0])
r = int(model_path.split('_')[3].split('.')[0][1:])
model = bidirec_LSTM(V, D, H, H_f, O, da, r, num_layers=num_layers, bidirec=bidirec, use_cuda=USE_CUDA)
if USE_CUDA:
    model = model.cuda()
    model.load_state_dict(torch.load(model_path))
else:
    model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage))

In [20]:
def save_data(i, model, inputs, lengths, targets, preds, dic):
    dic[i] = {'inputs': inputs, 'lengths': lengths, 'A': model.A, 'targets': targets, 'preds': preds}
    return dic

In [21]:
num_equal=0
dic = defaultdict(dict)
for i, batch in enumerate(test_iter):
    inputs, lengths = batch.review
    targets = batch.label
    if 0 in lengths:
        idxes = torch.arange(inputs.size(0))
        if USE_CUDA:
            idxes = idxes.cuda()
        mask = Variable(idxes[lengths.ne(0)].long())

        inputs = inputs.index_select(0, mask)
        lengths = lengths.masked_select(lengths.ne(0))
        targets = targets.index_select(0, mask)
    
    preds = model.predict(inputs, lengths)
    num_equal += torch.eq(preds, targets).sum().data[0]
    dic = save_data(i, model, inputs, lengths, targets, preds, dic)
print("Accuracy : " , num_equal / len(test_data))

Accuracy :  0.84604


/Users/Soo/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  app.launch_new_instance()


In [62]:
def get_reviews(inputs, lengths):
    max_len = max(lengths)
    reviews = []
    for s, l in zip(inputs, lengths):
        if l == max_len:
            reviews.append([REVIEW.vocab.itos[w.data[0]] for w in s])
        else:
            num_add_pad = max_len - l
            reviews.append([REVIEW.vocab.itos[w.data[0]] for w in s] + ['<pad>']*(num_add_pad-1))
    return reviews

In [63]:
def build_matrix(dic, sample_idx):
    A = dic[sample_idx]['A'].sum(1)
    A_norm = (A / A.max(1)[0].unsqueeze(1)).data.numpy()
    inputs = dic[sample_idx]['inputs']
    lengths = dic[sample_idx]['lengths']
    targets = dic[sample_idx]['targets']
    preds = dic[sample_idx]['preds']
    
    reviews = get_reviews(inputs, lengths)
    return reviews, A_norm, targets, preds

In [64]:
def span_str(color, input_str):
    string = '<span style="margin-left:3px;background-color:rgba({})">{}</span>'.format(color, input_str)
    return string

In [65]:
def write_html_vis(reviews, A_norm, targets, preds, vis_path):
    with open(vis_path, 'w', encoding='utf-8') as file:
        print('<html style="margin:0;padding:0;"><body style="margin:0;padding:0;">\n', file=file)
        print('<div style="margin:25px;">\n', file=file)
        for review, score, target, preds in zip(reviews, A_norm, \
                                                targets.data.numpy(), preds.data.numpy()):
            alphas = ['{:.2f}'.format(s) for s in score]
            print('<p style="margin:10px;">\n', file=file)
            if target == preds:
                color = '154,226,143, 0.7'
                input_str = '[target: {}| pred: {}]'
                string = span_str(color, input_str)
                print(string.format(target, preds), file=file)
            else:
                color = '203,127,230, 0.7'
                input_str = '[target: {}| pred: {}]'
                string = span_str(color, input_str)
                print(string.format(target, preds), file=file)
                
            for word, alpha in zip(review, alphas):
                color = '255,50,50, {}'
                input_str = '{}'
                string = span_str(color, input_str)
                print(('\t' + string + '\n').format(alpha, word), file=file)
            print('</p>\n', file=file)
        print('</div>\n', file=file)
        print('</body></html>', file=file)

In [66]:
# sample_idx = np.random.randint(len(dic))
sample_idx = 767

In [70]:
vis_path = '../figures/{}[{}].html'.format(model_path.split('/')[-1].split('.')[0], sample_idx)

In [71]:
reviews, A_norm, targets, preds = build_matrix(dic, sample_idx)

In [72]:
write_html_vis(reviews, A_norm, targets, preds, vis_path)